# Scraping Booking.com

In [ ]:
# Libraries to import
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import re
import json
import csv

In [ ]:
# Code for scraping Booking.com

# Initialize WebDriver
driver = webdriver.Chrome()
#driver.maximize_window()

base_url = 'https://www.booking.com/searchresults.it.html?ss='
comuni_interesse = [
    'COLLIO',
    'ROGNO',
    'VALDISOTTO',
    'OLTRE IL COLLE',
    'ANGOLO TERME',
    'SAN PELLEGRINO TERME',
    'VALTORTA',
    'ARTOGNE',
    'GROMO',
    'TEGLIO',
    'PIURO',
    'GEROLA ALTA',
    'SANTAMARGHERITADISTAFFORA',
    'APRICA',
    'MADESIMO',
    'MARGNO',
    'VALBONDIONE',
    'BAGOLINO',
    'BRENO',
    'PISOGNE',
    'BORNO',
    'BARZIO',
    'LECCO',
    'CORTENO GOLGI',
    'MOGGIO',
    'SCHILPARIO',
    'CAMPODOLCINO',
    'LANZADA',
    'TEMU',  # Temù
    'VALFURVA',
    'CASPOGGIO',
    'FOPPOLO',
    'CASTIONE DELLA PRESOLANA',
    'VALLEVE',
    'SELVINO',
    'BORMIO',
    'LIVIGNO',
    'PONTE DI LEGNO',
    'SONGAVAZZO',
    'CHIESA IN VALMALENCO',
    'CARONA',
    'COLERE',
    'VALDIDENTRO',
    'PIAZZATORRE'
]

# iterate over the "comune" list to create the urls
url_list = [base_url + comune.replace(" ", "%20") for comune in comuni_interesse]

result_dict = {}  # Initialize empty dictionariy
bh_data = []  # Initialize empty list

# iterate over the "comune" list to get the informations on the html page
for url in url_list:
    driver.get(url)
    wait = WebDriverWait(driver, 20)

    # close pop ups
    pop_ups = driver.find_elements(By.CLASS_NAME, 'pop_sharp_box')
    if pop_ups:
        for pop_up in pop_ups:
            close_button = pop_up.find_element(By.CLASS_NAME, 'close_button')
            close_button.click()
        time.sleep(2)

    # use Beautifulsoup to catch the page source
    soup = BeautifulSoup(driver.page_source, features="html.parser")

    # initialize WebDriverWait
    wait = WebDriverWait(driver, 40)

    # extract results info
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.fcab3ed991.d5f78961c3')))
        n_results = soup.find('h1', class_='fcab3ed991 d5f78961c3')
        if n_results:
            print(f'Primo: {n_results}')
        else:
            raise TimeoutException("Element not found with CSS selector 'h1.fcab3ed991.d5f78961c3'")
    except TimeoutException:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.e1f827110f.d5f78961c3')))
        n_results = soup.find('h1', class_='e1f827110f d5f78961c3')
        if n_results:
            print(f'Secondo: {n_results}')
        else:
            print("Element not found with CSS selector 'h1.e1f827110f.d5f78961c3'")

    if n_results:
        n_results_text = n_results.text
        colon_index = n_results_text.find(':')
        if colon_index != -1:
            comune = n_results_text[:colon_index].strip()
        else:
            comune = ''

        number = re.search(r'\d+', n_results_text)
        if number:
            result_number = int(number.group())
        else:
            result_number = 0
    else:
        comune = ''
        result_number = 0

    print(comune)
    time.sleep(2)

    scraped_hotels = 0   # intialize the count of the hotels scraped

    # if the number og hotel scraped is less than the result number catched on the fron page go on scraping
    while scraped_hotels < result_number:
        soup = BeautifulSoup(driver.page_source, features="html.parser")
        hotels = soup.find_all('div', class_='a826ba81c4 fa2f36ad22 afd256fc79 d08f526e0d ed11e24d01 ef9845d4b3 da89aeb942')

        for hotel in hotels:

            pop_ups = driver.find_elements(By.CLASS_NAME, 'pop_sharp_box')
            if pop_ups:
                for pop_up in pop_ups:
                    close_button = pop_up.find_element(By.CLASS_NAME, 'close_button')
                    close_button.click()
                time.sleep(10)

            # extract hotel info
            title_element = hotel.find('div', class_='fcab3ed991 a23c043802')
            title = title_element.text.strip() if title_element else ''

            link_element = hotel.find('a', class_='e13098a59f')['href']
            driver.get(link_element)
            soup2 = BeautifulSoup(driver.page_source, 'html.parser')

            indirizzo_element = soup2.find('span', class_='hp_address_subtitle')
            indirizzo = indirizzo_element.text.strip() if indirizzo_element else ''

            rating_element = soup2.find('div', class_='b5cd09854e d10a6220b4')
            rating = rating_element.text.strip() if rating_element else ''

            review_element = soup2.find('div', class_='d8eab2cf7f c90c0a70d3 db63693c62')
            review = review_element.text.strip() if review_element else ''

            try:
                staff_element = soup2.find('div', id=':Rhi:-label')
                staff = staff_element.text.strip()
            except AttributeError:
                staff_element = soup2.find('div', id=':R9i:-label')
                staff = staff_element.text.strip() if staff_element else ''

            try:
                servizi_element = soup2.find('div', id=':Rii:-label')
                servizi = servizi_element.text.strip()
            except AttributeError:
                servizi_element = soup2.find('div', id=':Rai:-label')
                servizi = servizi_element.text.strip() if servizi_element else ''

            try:
                pulizia_element = soup2.find('div', id=':Rji:-label')
                pulizia = pulizia_element.text.strip()
            except AttributeError:
                pulizia_element = soup2.find('div', id=':Rbi:-label')
                pulizia = pulizia_element.text.strip() if pulizia_element else ''

            try:
                comfort_element = soup2.find('div', id=':Rki:-label')
                comfort = comfort_element.text.strip()
            except AttributeError:
                comfort_element = soup2.find('div', id=':Rci:-label')
                comfort = comfort_element.text.strip() if comfort_element else ''

            try:
                rapporto_qualità_prezzo_element = soup2.find('div', id=':Rli:-label')
                rapporto_qualità_prezzo = rapporto_qualità_prezzo_element.text.strip()
            except AttributeError:
                rapporto_qualità_prezzo_element = soup2.find('div', id=':Rdi:-label')
                rapporto_qualità_prezzo = rapporto_qualità_prezzo_element.text.strip() if rapporto_qualità_prezzo_element else ''

            try:
                posizione_element = soup2.find('div', id=':Rmi:-label')
                posizione = posizione_element.text.strip()
            except AttributeError:
                posizione_element = soup2.find('div', id=':Rei:-label')
                posizione = posizione_element.text.strip() if posizione_element else ''

            wifi_gratuito_element = soup2.find('div', id=':Rni:-label')
            wifi_gratuito = wifi_gratuito_element.text.strip() if wifi_gratuito_element else ''

            driver.back()

            # create a dictionary where to save the info
            hotel_info = {
                'nome': title,
                'booking_link': link_element,
                'indirizzo': indirizzo,
                'rating': rating,
                'n_reviews': review,
                'staff': staff,
                'servizi': servizi,
                'pulizia': pulizia,
                'comfort': comfort,
                'rapporto qualità prezzo': rapporto_qualità_prezzo,
                'posizione': posizione,
                'wifi gratuito': wifi_gratuito,
                'comune': comune
            }

            bh_data.append(hotel_info)

            scraped_hotels += 1
            time.sleep(2)

            if scraped_hotels >= result_number:
                break

            # every page contains 25 hotels, if the result number is not reached butyou reached 25, go to the next page.
            if scraped_hotels < result_number:
                if result_number > 25:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    try:
                        button = driver.find_element(By.XPATH, '//*[@id="search_results_table"]/div[2]/div/div/div[4]/div[2]/nav/div/div[3]/button')
                        button.click()
                    except NoSuchElementException:
                        wait = WebDriverWait(driver, 20)
                        wait.until(EC.invisibility_of_element_located((By.ID, "onetrust-banner-sdk")))
                        wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.otFlat.bottom.vertical-align-content.ot-buttons-fw')))
                        button = driver.find_element(By.XPATH, '//*[@id="search_results_table"]/div[2]/div/div/div[4]/div[2]/nav/div/div[3]/button')
                        button.click()

                else:
                    continue


    # Save to CSV
    csv_fields = ['nome','booking_link', 'indirizzo', 'rating', 'n_reviews', 'staff', 'servizi', 'pulizia', 'comfort', 'rapporto qualità prezzo', 'posizione', 'wifi gratuito', 'comune']
    with open('booking.csv', 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=csv_fields)
        writer.writeheader()
        writer.writerows(bh_data)

# Close the driver
driver.quit()

print(bh_data)

Primo: <h1 aria-label="Ora i risultati sono aggiornati. Collio: 2 strutture trovate." aria-live="assertive" class="fcab3ed991 d5f78961c3">Collio: 2 strutture trovate</h1>
Collio
Primo: <h1 aria-label="Ora i risultati sono aggiornati. Rogno: nessuna struttura trovata." aria-live="assertive" class="fcab3ed991 d5f78961c3">Rogno: nessuna struttura trovata</h1>
Rogno
Primo: <h1 aria-label="Ora i risultati sono aggiornati. Valdisotto: 44 strutture trovate." aria-live="assertive" class="fcab3ed991 d5f78961c3">Valdisotto: 44 strutture trovate</h1>
Valdisotto
Primo: <h1 aria-label="Ora i risultati sono aggiornati. Oltre il Colle: 3 strutture trovate." aria-live="assertive" class="fcab3ed991 d5f78961c3">Oltre il Colle: 3 strutture trovate</h1>
Oltre il Colle
Primo: <h1 aria-label="Ora i risultati sono aggiornati. Angolo Terme: 5 strutture trovate." aria-live="assertive" class="fcab3ed991 d5f78961c3">Angolo Terme: 5 strutture trovate</h1>
Angolo Terme
Primo: <h1 aria-label="Ora i risultati sono a

In [ ]:
df = pd.DataFrame(bh_data)